In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')


# custom functions for pulling
from db_functions import *

pd.set_option('display.max_columns', None)

# Player Database

We start by loading our data and removing some variables we don't need. 

In [2]:
df = pd.read_csv('FIFA19 - Ultimate Team players.csv', low_memory = False)
cols_drop = ['origin', 'player_extended_name', 'price_xbox', 'price_pc', 'specialties', 'cb', 'lb', 'rb', 'rwb', 'lwb', 'cdm', 'cm', 'cam', 'lm', 'lw', 'lf', 'rm', 'rw', 'rf', 'cf', 'st', 
                   'price_ps4', 'gk_positoning', 'gk_kicking', 'gk_speed', 'gk_handling', 'gk_reflexes', 'gk_diving', 'date_of_birth', 'traits']
df.drop(cols_drop, axis = 1, inplace = True)
df = df[(df.quality == 'Gold - Rare') | (df.quality == 'Gold')]
df.reset_index(drop = True, inplace = True)
df['resource_id'] = np.nan
df['id'] = df.player_ID
df.set_index('id', inplace = True)

In [3]:
df.tail(2)

,player_ID,player_name,quality,revision,overall,club,league,nationality,position,age,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,resource_id
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17020,17020,Gelson Martins,Gold - Rare,SBC,82,Atlético Madrid,LaLiga Santander,Portugal,RM,23,173,72,3,2018-10-05,94.0,95,94,86.0,94,92,83,83,85,85,69.0,82,66,72,69,74,66,75.0,79,81,60,78,55,82,47.0,43,51,54,46,36,61.0,65,87,48,60,Right,High,Med,3,5,NaN
17021,17021,Verdi,Gold - Rare,OTW,81,Napoli,Serie A TIM,Italy,RW,26,171,61,2,2018-10-05,87.0,88,86,85.0,92,90,78,86,84,78,73.0,76,67,81,77,74,68,81.0,82,82,84,82,76,84,48.0,36,42,55,48,60,55.0,51,75,48,48,Left,High,Med,5,4,NaN


In [4]:
print('As of now, we have {} players in our database and no prices.'.format(df.shape[0]))
print('Our data consists of {} features.'.format(df.shape[1]))

As of now, we have 2920 players in our database and no prices.
Our data consists of 55 features.


# Data Mining

We wish to add all the gold players (including special cards) along with daily prices. To do so, we need to do the following:
1. Scrape futbin and collect the resource id for every player listed above, and add that to our dataframe.
2. Build a script that adds all the new players that are not in our dataframe (collects all relevant data including the resource id). 
3. Collect the prices on all the players in our database using the resource id. Construct multiple observations for each player, one for each price-point. 


#### Step 1

To scrape the resource id, we will be using some functions defined below that use requests and beautiful soup to parse the html of each player's page on futbin.

In [6]:
df_p = df_fetch_resourceid(df)

Turns out, we also need the PGP data for each player, so we parse that too. 

In [7]:
df_p = df_p.groupby('player_ID').last()
df_p['num_games'] = np.nan
df_p['avg_goals'] = np.nan
df_p['avg_assists'] = np.nan

In [8]:
df_players = df_fetch_pgp(df_players)

#### Step 2

We have collected the resource_id for the players in our dataframe. To proceed, we're going to write some functions to collect data on players that are not in our dataframe and add them. 

In [9]:
df_p = df_fetch_newplayers(17503, df_p)

Before we move on to step 3, we should filter our constructed dataframe s.t we only keep the players we are actually interested in.
We remove:
- Goalkeepers
- Silver players
- Bronze players

There are various players that are not unavailable to be purchased, e.g. Flashback SBC players, but those will be dealt with later.

In [12]:
df_p = df_p[(df_p.quality == 'Gold - Rare') | (df_p.quality == 'Gold') | (df_p.quality == 'gold rare')]
df_p['revision'] = df_p.revision.fillna('Normal')
df_p = df_p[df_p.position != 'GK']

#### Player Database is Ready

We save the player database before proceeding to getting the prices for each player.

In [13]:
df_p.to_csv('player_database.csv')

We now fetch the prices for each player in our database. This process will probably take long, depending on the size of the player database. 

In [14]:
df_prices = df_fetch_price(df_p)

In [15]:
df_prices.to_csv('prices_database.csv')

# Dataframe Update

To update our dataframe when new players are added to the game, we do the following:
- Read in our player dataframe
- Scrape the data for all the new players
- Fetch their prices

In [2]:
df_p = pd.read_csv('player_database.csv', index_col='player_ID', parse_dates=['added_date'])
# df_p.drop('player_ID.1', inplace=True, axis=1)
df_p = df_fetch_newplayers(18735, df_p)

Completed 1 players. Time elapsed: 0 seconds.
Approximate time left: 0 seconds.
No player found at ID: 18180.
No player found at ID: 18193.
No player found at ID: 18209.
No player found at ID: 18215.
No player found at ID: 18233.
No player found at ID: 18243.
No player found at ID: 18249.
No player found at ID: 18261.
No player found at ID: 18276.
No player found at ID: 18281.
No player found at ID: 18284.
No player found at ID: 18299.
No player found at ID: 18312.
No player found at ID: 18319.
No player found at ID: 18334.
No player found at ID: 18339.
No player found at ID: 18351.
No player found at ID: 18353.
Completed 200 players. Time elapsed: 218 seconds.
Approximate time left: 392 seconds.
No player found at ID: 18378.
No player found at ID: 18386.
No player found at ID: 18389.
No player found at ID: 18400.
No player found at ID: 18407.
No player found at ID: 18438.
No player found at ID: 18465.
No player found at ID: 18466.
No player found at ID: 18506.
No player found at ID: 1

In [3]:
df_p.tail()

,player_name,quality,revision,overall,club,league,nationality,position,age,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,resource_id,num_games,avg_goals,avg_assists
player_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18731,Scott Wagstaff,gold rare,MOTM,75,AFC Wimbledon,EFL League One,England,CM,28,178,79,1,2019-02-03,84.0,87,81,75.0,90,82,70,72,75,63,70.0,82,73,73,59,70,57,70.0,71,78,50,67,68,73,70.0,70,57,71,72,70,73.0,88,88,71,54,Right,High,Med,4,2,50518213,24,0.13,0.25
18732,Leighton Baines,gold rare,Flashback SBC,87,Everton,Premier League,England,LB,34,170,70,3,2019-02-03,84.0,88,81,82.0,72,87,85,84,80,88,79.0,81,70,93,79,70,99,87.0,83,92,89,87,81,89,83.0,85,78,81,84,83,81.0,83,84,73,92,Left,High,Med,3,3,50495279,315,0.06,0.07
18733,Klaas-Jan Huntelaar,gold rare,Flashback SBC,89,Ajax,Eredivisie,Holland,ST,35,186,83,3,2019-02-04,80.0,74,84,83.0,72,77,99,86,77,99,89.0,95,91,91,83,92,70,73.0,76,73,76,78,50,83,40.0,30,90,36,37,26,90.0,99,68,99,88,Right,Med,Med,4,2,67257667,7,0.86,0.14
18734,Martín Cáceres,gold rare,Normal,78,Juventus,Serie A TIM,Uruguay,CB,31,180,78,2,2019-02-06,76.0,78,74,66.0,79,74,73,66,60,76,51.0,50,37,70,59,65,50,64.0,58,69,41,67,65,68,79.0,80,76,80,78,77,73.0,91,69,72,78,Right,Med,High,3,2,67291359,-,-,-
18735,Gonzalo Higuaín,gold rare,Normal,88,Chelsea,Premier League,Argentina,ST,31,186,89,4,2019-02-06,73.0,73,73,83.0,75,69,86,85,84,86,87.0,92,92,86,80,90,70,70.0,74,68,62,75,59,74,31.0,20,80,35,22,18,74.0,79,70,85,50,Right,High,Med,4,3,67276528,-,-,-


In [4]:
df_p = df_p[(df_p.quality == 'Gold - Rare') | (df_p.quality == 'Gold') | (df_p.quality == 'gold rare')]
df_p['revision'] = df_p.revision.fillna('Normal')
df_p = df_p[df_p.position != 'GK']
df_p.to_csv('player_database.csv')

In [ ]:
df_prices = df_fetch_price(df_p)

In [ ]:
df_prices.tail()

In [ ]:
df_prices.to_csv('prices_database.csv')